In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Rio Grande Do Sul - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t),Rio Grande Do Sul - Consumo de Cimento (t)
0,2003,4.719016,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,20.534784,...,73.424505,1.689576,0.775641,8.192273,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,1576.618236,2103.000000
1,2004,4.572893,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,19.331964,...,73.472109,1.648349,0.776178,8.121313,2.093709e+08,8.969484e+06,18.258966,1.967326e+08,1530.612911,2116.716000
2,2005,4.580017,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,19.728565,...,73.731114,1.630458,0.776738,8.096895,2.119422e+08,9.019560e+06,18.316116,1.973689e+08,1526.253731,1966.059000
3,2006,4.941856,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,20.254598,...,74.009463,1.631146,0.777325,8.011393,2.145089e+08,9.086122e+06,18.398786,1.982320e+08,1625.631583,2106.926000
4,2007,5.580982,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,20.801125,...,74.316933,1.628793,0.777940,7.986478,2.168705e+08,9.162867e+06,18.479506,1.990254e+08,1786.663086,2280.943000
5,2008,5.988218,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,21.764081,...,75.001024,1.589194,0.778585,7.956795,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,1892.120992,2718.542000
6,2009,6.131171,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,22.056812,...,75.640304,1.544445,0.779263,7.985037,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,1987.964633,2685.740000
7,2010,6.216180,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,21.905564,...,75.347633,1.511781,0.779976,7.949439,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,2093.049582,3070.312000
8,2011,6.262114,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,22.168556,...,74.832405,1.485231,0.780726,7.936698,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,2163.054486,3399.177000
9,2012,6.449831,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,22.605974,...,75.069595,1.460267,0.781171,6.613682,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,2145.073736,3487.789000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,4.719016,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,20.534784,1010.304652,...,14.560522,73.424505,1.689576,0.775641,8.192273,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,1576.618236
1,4.572893,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,19.331964,1010.323155,...,14.560061,73.472109,1.648349,0.776178,8.121313,2.093709e+08,8.969484e+06,18.258966,1.967326e+08,1530.612911
2,4.580017,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,19.728565,1010.259871,...,14.607303,73.731114,1.630458,0.776738,8.096895,2.119422e+08,9.019560e+06,18.316116,1.973689e+08,1526.253731
3,4.941856,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,20.254598,1010.152487,...,14.544771,74.009463,1.631146,0.777325,8.011393,2.145089e+08,9.086122e+06,18.398786,1.982320e+08,1625.631583
4,5.580982,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,20.801125,1009.972382,...,14.420724,74.316933,1.628793,0.777940,7.986478,2.168705e+08,9.162867e+06,18.479506,1.990254e+08,1786.663086
5,5.988218,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,21.764081,1009.667623,...,14.577965,75.001024,1.589194,0.778585,7.956795,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,1892.120992
6,6.131171,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,22.056812,1009.481089,...,14.707061,75.640304,1.544445,0.779263,7.985037,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,1987.964633
7,6.216180,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,21.905564,1009.521141,...,14.610253,75.347633,1.511781,0.779976,7.949439,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,2093.049582
8,6.262114,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,22.168556,1009.586912,...,14.612190,74.832405,1.485231,0.780726,7.936698,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,2163.054486
9,6.449831,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,22.605974,1009.662812,...,14.847869,75.069595,1.460267,0.781171,6.613682,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,2145.073736


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2116.716000
1     1966.059000
2     2106.926000
3     2280.943000
4     2718.542000
5     2685.740000
6     3070.312000
7     3399.177000
8     3487.789000
9     3505.720000
10    3226.786714
11    2978.617371
12    2744.168914
13    2703.746000
14    2665.575000
15    2753.564000
16    2993.250000
17    3222.295000
18    3135.029000
19            NaN
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,-1.436522,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.036234,1.694623,...,-0.864486,-1.301149,1.788718,-1.632255,1.182346,-1.618491,-1.821147,-2.128488,-1.958357,-0.701822
1,-1.570347,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.693339,1.767185,...,-0.865255,-1.275523,1.306887,-1.453138,1.015589,-1.421437,-1.362802,-1.819374,-1.670491,-0.885458
2,-1.563823,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.476674,1.519003,...,-0.786459,-1.136092,1.097798,-1.266062,0.958207,-1.236614,-0.994708,-1.479737,-1.397152,-0.902858
3,-1.232437,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.189300,1.097875,...,-0.890758,-0.986247,1.105843,-1.070292,0.757276,-1.052126,-0.505440,-0.988435,-1.026379,-0.506179
4,-0.647100,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.890730,0.391557,...,-1.097659,-0.820726,1.078338,-0.865094,0.698725,-0.882374,0.058684,-0.508726,-0.685546,0.136598
5,-0.274137,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.364664,-0.803615,...,-0.835393,-0.452456,0.615543,-0.649732,0.628969,-0.743570,0.567715,-0.237746,-0.488677,0.557546
6,-0.143215,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.204744,-1.535147,...,-0.620069,-0.108311,0.092552,-0.423472,0.695339,-0.634462,1.009311,-0.041436,-0.352848,0.940117
7,-0.065361,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.287372,-1.378078,...,-0.781538,-0.265865,-0.289198,-0.185580,0.611681,-0.238315,1.506834,0.439197,0.162461,1.359577
8,-0.023293,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.143698,-1.120144,...,-0.778307,-0.543229,-0.599493,0.064679,0.581739,0.113822,1.587817,0.726199,0.494609,1.639010
9,0.148625,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.095265,-0.822486,...,-0.385212,-0.415542,-0.891244,0.213398,-2.527373,0.436677,1.440210,0.870870,0.776819,1.567237


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2116.716000
1     1966.059000
2     2106.926000
3     2280.943000
4     2718.542000
5     2685.740000
6     3070.312000
7     3399.177000
8     3487.789000
9     3505.720000
10    3226.786714
11    2978.617371
12    2744.168914
13    2703.746000
14    2665.575000
15    2753.564000
16    2993.250000
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
17,2.125084,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.450579,-0.347959,...,0.230069,0.172136,-0.312972,0.083121,-1.130369,0.714993,-0.725876,-0.138731,0.085128,-0.794517


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    3222.295
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
14,1.029414,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.260387,0.087895,...,1.823189,1.844947,-0.074849,1.310880,-0.669543,1.190613,-0.739637,0.674936,0.853475,-1.182509
15,1.325631,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.664667,0.154053,...,1.392026,1.179035,0.004926,1.051749,-0.856070,1.152075,-0.716561,0.589431,0.744797,-1.341464
16,1.830538,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,0.565321,-0.066070,...,0.625804,0.328189,-0.166436,0.647697,-0.933365,1.004011,-0.626306,0.280742,0.476893,-1.155978


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2506649421, 1100849130, 2000557400, 1055068423, 4249161502, 3482775947, 965834328, 123247791, 737479113, 3897884063, 4209287469, 3473638656, 2264158007, 10973250, 2609811195, 2326398933, 4025833718, 4147711821, 430115247, 2832782165, 4055175021, 2775138323, 4084091240, 3931285644, 1795856508, 1996922018, 2410480729, 4141480068, 879703079, 203978774, 3887238964, 3629816304, 3801728360, 1305510899, 480202140, 2769320330, 1480040736, 3195628837, 315133600, 2354473128, 1057118524, 3173649739, 1118660837, 1376836247, 425086087, 2245196343, 3265371154, 931693118, 2864567440, 2499313601, 1621056163, 1059225935, 2255117639, 3609990936, 3007952481, 4142386244, 3837593330, 1588745023, 4208395653, 4182898870, 4139841507, 1200035010, 3821466280, 267099059, 3527348491, 2772592998, 3204957, 1723069416, 339452751, 686249411, 166797572, 3753911562, 2276629503, 3047013514, 2733713452, 1047289765, 953903633, 1118638977, 433846815, 2812449687, 961005247, 1147051171, 3177278432, 39079016, 3720414067, 103

val_loss: 12566.0498046875


Step: 86 ___________________________________________
val_loss: 19175.5234375


Step: 87 ___________________________________________
val_loss: 13056.9169921875


Step: 88 ___________________________________________
val_loss: 9655.5283203125


Step: 89 ___________________________________________
val_loss: 6265.91259765625


Step: 90 ___________________________________________
val_loss: 11300.9921875


Step: 91 ___________________________________________
val_loss: 19173.8203125


Step: 92 ___________________________________________
val_loss: 19173.982421875


Step: 93 ___________________________________________
val_loss: 19177.087890625


Step: 94 ___________________________________________
val_loss: 1479.2459716796875


Step: 95 ___________________________________________
val_loss: 19173.6015625


Step: 96 ___________________________________________
val_loss: 12678.2431640625


Step: 97 ___________________________________________
val_loss: 14849.78515625


St

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 478ms/step - loss: 7987680.0000 - val_loss: 7528270.5000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 7686209.0000 - val_loss: 411909.7812
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 13366304.0000 - val_loss: 4678461.5000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 6195164.0000 - val_loss: 2087369.3750
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 5955328.5000 - val_loss: 1882066.1250
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 6066492.5000 - val_loss: 3406784.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 6832544.0000 - val_loss: 6538036.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 6563303.5000 - val_loss: 4549720.5000
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 6074170.5000 - val_loss: 5085

1/1 [==============================] - 0s 14ms/step - loss: 1539175.6250 - val_loss: 4391587.0000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1548536.2500 - val_loss: 2837790.7500
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1572575.5000 - val_loss: 3688723.0000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1544661.7500 - val_loss: 3136567.7500
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1305822.2500 - val_loss: 3530221.7500
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 1825115.1250 - val_loss: 3174235.0000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1646082.7500 - val_loss: 2456219.0000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1450025.0000 - val_loss: 2601405.0000
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 1657065.5000 - val_loss: 2342816.750

1/1 [==============================] - 0s 14ms/step - loss: 522353.5938 - val_loss: 932939.6875
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 530755.1250 - val_loss: 1051508.8750
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 427238.5312 - val_loss: 831754.3125
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 471304.5938 - val_loss: 1018772.3125
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 379052.6250 - val_loss: 765039.8125
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 584016.6875 - val_loss: 823667.0000
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 472414.7500 - val_loss: 1755850.6250
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 638316.5000 - val_loss: 903249.5000
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 517817.4688 - val_loss: 764488.7500
Epoch

1/1 [==============================] - 0s 14ms/step - loss: 180395.5781 - val_loss: 362685.8438
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 110790.1016 - val_loss: 382097.2188
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 207910.1719 - val_loss: 355835.9062
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 124265.5859 - val_loss: 370717.5000
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 120308.1875 - val_loss: 374075.2188
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 165040.3750 - val_loss: 417037.0938
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 153765.3750 - val_loss: 298219.9688
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 227153.4062 - val_loss: 256283.0781
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 191991.2656 - val_loss: 356399.6250
Epoch 22

1/1 [==============================] - 0s 14ms/step - loss: 86248.3438 - val_loss: 75672.2422
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 110825.9688 - val_loss: 172995.7656
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 87411.9375 - val_loss: 64247.6406
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 110747.5312 - val_loss: 109390.9375
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 94443.9062 - val_loss: 143285.3594
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 69878.5312 - val_loss: 166824.4375
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 79115.9844 - val_loss: 90226.1875
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 99790.2109 - val_loss: 135978.7500
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 99157.3828 - val_loss: 165314.2031
Epoch 300/10000
1/

...

1/1 [==============================] - 0s 14ms/step - loss: 56175.7734 - val_loss: 6957.3398
Epoch 1181/10000
1/1 [==============================] - 0s 13ms/step - loss: 45157.2188 - val_loss: 4817.1196
Epoch 1182/10000
1/1 [==============================] - 0s 13ms/step - loss: 48345.5000 - val_loss: 5698.3237
Epoch 1183/10000
1/1 [==============================] - 0s 14ms/step - loss: 55839.4414 - val_loss: 3313.5544
Epoch 1184/10000
1/1 [==============================] - 0s 13ms/step - loss: 46384.2539 - val_loss: 3273.8982
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 56934.1016 - val_loss: 7704.7563
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 68602.2109 - val_loss: 8322.8525
Epoch 1187/10000
1/1 [==============================] - 0s 14ms/step - loss: 82770.0859 - val_loss: 5398.0459
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 35748.9492 - val_loss: 2916.4094
Epoch 1189/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 67527.6641 - val_loss: 5850.9844
Epoch 1255/10000
1/1 [==============================] - 0s 14ms/step - loss: 59282.9453 - val_loss: 9191.5098
Epoch 1256/10000
1/1 [==============================] - 0s 14ms/step - loss: 88053.5234 - val_loss: 6280.0659
Epoch 1257/10000
1/1 [==============================] - 0s 13ms/step - loss: 60040.8086 - val_loss: 7605.9121
Epoch 1258/10000
1/1 [==============================] - 0s 14ms/step - loss: 44454.4062 - val_loss: 13063.9854
Epoch 1259/10000
1/1 [==============================] - 0s 13ms/step - loss: 83365.9531 - val_loss: 5949.0591
Epoch 1260/10000
1/1 [==============================] - 0s 14ms/step - loss: 79214.0625 - val_loss: 2187.1919
Epoch 1261/10000
1/1 [==============================] - 0s 14ms/step - loss: 48244.9688 - val_loss: 2763.3347
Epoch 1262/10000
1/1 [==============================] - 0s 14ms/step - loss: 97386.4453 - val_loss: 2238.9236
Epoch 1263/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 80191.5391 - val_loss: 57389.2227
Epoch 1329/10000
1/1 [==============================] - 0s 14ms/step - loss: 67007.4453 - val_loss: 6295.8735
Epoch 1330/10000
1/1 [==============================] - 0s 13ms/step - loss: 52664.2422 - val_loss: 6825.7446
Epoch 1331/10000
1/1 [==============================] - 0s 14ms/step - loss: 65758.3828 - val_loss: 3905.9795
Epoch 1332/10000
1/1 [==============================] - 0s 13ms/step - loss: 71861.4531 - val_loss: 11877.2412
Epoch 1333/10000
1/1 [==============================] - 0s 13ms/step - loss: 81815.4219 - val_loss: 8582.6328
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 75717.4766 - val_loss: 9113.2148
Epoch 1335/10000
1/1 [==============================] - 0s 13ms/step - loss: 56238.5391 - val_loss: 25974.6934
Epoch 1336/10000
1/1 [==============================] - 0s 13ms/step - loss: 58520.2891 - val_loss: 6316.9312
Epoch 1337/10000
1/1 [==

1/1 [==============================] - 0s 14ms/step - loss: 42364.8789 - val_loss: 32717.8652
Epoch 1403/10000
1/1 [==============================] - 0s 15ms/step - loss: 56721.0391 - val_loss: 22192.9531
Epoch 1404/10000
1/1 [==============================] - 0s 14ms/step - loss: 77905.3516 - val_loss: 20030.3672
Epoch 1405/10000
1/1 [==============================] - 0s 14ms/step - loss: 75374.0078 - val_loss: 27072.1816
Epoch 1406/10000
1/1 [==============================] - 0s 14ms/step - loss: 56817.1875 - val_loss: 16292.7383
Epoch 1407/10000
1/1 [==============================] - 0s 14ms/step - loss: 77766.7891 - val_loss: 14031.3477
Epoch 1408/10000
1/1 [==============================] - 0s 14ms/step - loss: 46537.6953 - val_loss: 16662.5488
Epoch 1409/10000
1/1 [==============================] - 0s 14ms/step - loss: 60209.2891 - val_loss: 14367.2344
Epoch 1410/10000
1/1 [==============================] - 0s 14ms/step - loss: 43452.3789 - val_loss: 10930.6445
Epoch 1411/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 53034.8438 - val_loss: 26139.6113
Epoch 1477/10000
1/1 [==============================] - 0s 13ms/step - loss: 73764.1875 - val_loss: 10637.0166
Epoch 1478/10000
1/1 [==============================] - 0s 14ms/step - loss: 65135.1562 - val_loss: 28152.6641
Epoch 1479/10000
1/1 [==============================] - 0s 14ms/step - loss: 39963.0391 - val_loss: 12737.0205
Epoch 1480/10000
1/1 [==============================] - 0s 14ms/step - loss: 54919.4922 - val_loss: 24226.7344
Epoch 1481/10000
1/1 [==============================] - 0s 14ms/step - loss: 80995.7656 - val_loss: 16381.3555
Epoch 1482/10000
1/1 [==============================] - 0s 14ms/step - loss: 77892.6484 - val_loss: 12410.4766
Epoch 1483/10000
1/1 [==============================] - 0s 14ms/step - loss: 42834.9258 - val_loss: 9968.4014
Epoch 1484/10000
1/1 [==============================] - 0s 13ms/step - loss: 46445.3516 - val_loss: 20791.0957
Epoch 1485/10000
1/

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[3222.295](test_target) - [[2683.7627]](prediction) = 538.5323046875001


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,-0.929955,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.272794,1.035190,...,-0.174397,-1.169137,1.486007,-1.456068,1.870395,-1.525433,-1.476097,-1.507083,-1.534207,-0.853084
1,-1.148326,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.523246,1.090812,...,-0.180552,-1.110141,0.723617,-1.077568,1.013119,-1.055807,-1.060634,-1.143951,-1.105937,-1.076397
2,-1.137680,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-1.110939,0.900568,...,0.450274,-0.789155,0.392780,-0.682249,0.718127,-0.615330,-0.726979,-0.744962,-0.699280,-1.097557
3,-0.596933,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.564075,0.577751,...,-0.384727,-0.444195,0.405511,-0.268558,-0.314828,-0.175651,-0.283486,-0.167804,-0.147667,-0.615170
4,0.358203,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.004095,0.036321,...,-2.041135,-0.063147,0.361989,0.165057,-0.615828,0.228909,0.227860,0.395736,0.359404,0.166488
5,0.966793,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,1.005183,-0.879842,...,0.058514,0.784653,-0.370280,0.620148,-0.974436,0.559713,0.689266,0.714070,0.652295,0.678389
6,1.180428,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.309507,-1.440600,...,1.782357,1.576916,-1.197795,1.098269,-0.633239,0.819741,1.089547,0.944685,0.854373,1.143620
7,1.307470,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.152269,-1.320198,...,0.489667,1.214207,-1.801830,1.600970,-1.063310,1.763856,1.540523,1.509310,1.621018,1.653710


train_target:


,Rio Grande Do Sul - Consumo de Cimento (t)
0,2116.716
1,1966.059
2,2106.926
3,2280.943
4,2718.542
5,2685.740
6,3070.312
7,3399.177


test_input:


...

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
17,2.125084,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.450579,-0.347959,...,0.230069,0.172136,-0.312972,0.083121,-1.130369,0.714993,-0.725876,-0.138731,0.085128,-0.794517


test_target:


,Rio Grande Do Sul - Consumo de Cimento (t)
17,3222.295


1/1 [==============================] - 0s 35ms/step
Error: 538.5323046875001




[2971.146240234375,
 2753.357177734375,
 3243.1337890625,
 3366.277099609375,
 3191.63525390625,
 3061.271484375,
 2519.21923828125,
 2646.01513671875,
 2741.5712890625,
 2683.7626953125]

In [24]:
display(targets)
display(predictions)

[3487.789,
 3505.72,
 3226.7867142857144,
 2978.6173714285715,
 2744.1689142857144,
 2703.746,
 2665.575,
 2753.564,
 2993.25,
 3222.295]

[2971.146240234375,
 2753.357177734375,
 3243.1337890625,
 3366.277099609375,
 3191.63525390625,
 3061.271484375,
 2519.21923828125,
 2646.01513671875,
 2741.5712890625,
 2683.7626953125]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

352.21198496093746

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.11631254904343528